In [30]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
from bs4 import SoupStrainer as strainer
import re
import pprint

url = 'http://www.zgcjpx.com/cpa/tiku/lianxi/gszl/120156.html'
# url = 'http://www.zgcjpx.com/cpa/tiku/lianxi/jjf/130097.html'
# url = 'http://www.zgcjpx.com/cpa/tiku/lianxi/sj/120447.html'
with urlopen(url) as response:
    html = response.read()
page = soup(html, 'html.parser')

# filter out unwanted annoying <p>, and the rest would be the darling we love
def drills_para(tag):
    return tag.name == 'p' and len(tag.contents) <= 1 and isinstance(tag.contents[0], str)

drills = page.find_all(drills_para)

# delete yet another two
del drills[0]   # always be <p>CPA</p>
del drills[-1]  # always be <p>400-605-3513</p>

for i in range(len(drills)):
    drills[i] = drills[i] = drills[i].get_text().replace('\u3000', ' ')

    
# normal length == 25
print(len(drills))
print(drills)

# abnormal length != 25


# ------------------- workflow --------------------
# Single Question Type in Headline 2
# ------------------- headline2 -------------------



# que_type_pattern = re.compile(r'^[单多项选择题]+$')
# chp_info_pattern = re.compile('\(第.+?章.*?\)')

# questions_index = (0, 5, 10, 15, 20)

# que_types = []
# chap_info = []

# try:
#     if page.h2 != None and que_type_pattern.match(page.h2.get_text()):
#         print('h2 Text is correct: ', page.h2.get_text())
#         que_types.append(page.h2.get_text())
#     else:
#         raise ValueError
# except ValueError:
#     print('<h2> does not contain question type:', page.h2)


# for i in questions_index:    
#     # record chp info with "()" stripped
#     chap_info.append(chp_info_pattern.search(drills[i]).group().strip('()'))
#     drills[i] = re.sub(chap_info_pattern, '', drills[i])

# print(chap_info)

25
['1.泰华公司主要经营电子产品，但是目前该行业竞争激烈，利润空间狭窄，为了使企业能够长期稳定发展，公司决定进军日化行业，这体现了( )。(第一章 战略与战略管理)', 'A.公司宗旨', 'B.经营哲学', 'C.公司目的', 'D.公司目标', '2.甲公司注意到我国农村人口在过去习惯以麦秸秆烧火做饭，现在逐渐采用煤气、电力等烧火做饭，麦秸秆成了随处乱扔的废物。由此，甲公司决定进入生物质能源转化领域，以麦秸秆为原料进行电力的转化。这表明该企业注意到了环境中的(  )，从而做出了正确的战略选择。(第二章 战略分析)', 'A.政治和法律因素', 'B.经济因素', 'C.社会和文化因素', 'D.技术因素', '3.下列关于企业稳定战略的表述，正确的是( )。(第三章 战略选择)', 'A.采用稳定战略的企业容易减弱风险意识，降低对风险的敏感性', 'B.企业无法充分利用原有生产经营领域的资源', 'C.采用稳定战略的企业仍然需要投入一定的资金开发新产品和新市场', 'D.企业集中于原有的经营范围和产品，无法增加竞争优势', '4.甲公司是一家关注于高科技移动领域的互联网公司。公司没有森严的等级制度，强调员工平等，崇尚创新，在处理多样化的问题时，鼓励员工跨部门合作，在工作中发挥自己的专长和创意，努力打造客户需要的产品。甲公司的企业文化类型属于( )。(第四章 战略实施)', 'A.权力导向型', 'B.角色导向型', 'C.人员导向型', 'D.任务导向型', '5.信息披露制度的主要特征不包括( )。(第五章 公司治理)', 'A.信息披露义务的强制性和自愿性', 'B.信息披露内容的多样性', 'C.信息披露时间的持续性', 'D.信息披露在内部治理结构中起到监督作用']


In [ ]:
# ---------------- mixed question types -----------------

def unique_list(a_list):
    for item in a_list:
        while a_list.count(item) > 1:
            a_list.remove(item)
    return a_list


# ------------------- strange options ---------------------
# to catch strange options which do not start with any of the below prefix
prefix = ('A', 'B', 'C', 'D', '1', '2', '3', '4', '5')
unwanted = []
    

# patterns for use in regular expressions
# extract (单选题) or (多选题) and chapter info like (第六章.风险与风险管理)
# this is necessary since later they will get in the way of processing questiona-and-options
que_type_pattern = re.compile('[\(【][单多]选题?[\)】]')
# que_type_pattern = r'[\(【][单多项选择题]+[\)】]'
chp_info_pattern = re.compile('\(第.+?章.*?\)')
questions_index = (0, 5, 10, 15, 20)
que_type = []
chp_info = []